<a href="https://colab.research.google.com/github/plklinh/DATA501_Project/blob/main/Market_Forces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/My\ Drive/U\ of\ C/Data\ 501/DATA501_Project

/content/drive/My Drive/U of C/Data 501/DATA501_Project


In [4]:
import pandas as pd
import numpy as np
import datetime

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [10]:
def convert_datetime(df):
    df["Date"] = pd.to_datetime(df["Date"]) 

def set_datetime_index(df):
    df.set_index("Date", inplace=True)
    df.set_index( pd.to_datetime(df.index), inplace= True)

def split_train_test(df, perc):
    cut_off = int (np.round( df.shape[0] * perc ))
    return df[:cut_off], df[cut_off:]

In [11]:
hose = pd.read_csv("cleaned data/VNINDEX_2000_2020.csv")
hnx = pd.read_csv("cleaned data/HNX_2000_2020.csv")
set_datetime_index(hose)
set_datetime_index(hnx)

**Time Trends**

# Relationship with Foreign Markets 

In [9]:
def calc_trade_corr(vn_si_df, partner_si):
    partner_si_df = pd.DataFrame(partner_si) 
    trade_comp =partner_si_df.merge(vn_si_df, on = "Date" ,
                    how ="right")
    trade_comp.dropna(inplace=True)
    trade_return_comp = trade_comp.pct_change()
    return trade_return_comp.corr()

def make_garch_df(vn_si, partner_si):
    partner_si_df = pd.DataFrame(partner_si) 
    trade_comp =partner_si_df.merge(vn_si, on = "Date" ,
                    how ="inner")   
    trade_comp.dropna(inplace=True)
    trade_comp.set_index("Date", inplace=True)
    trade_comp.set_index( pd.to_datetime(trade_comp.index), inplace= True)
    trade_comp_log = np.log(trade_comp[[vn_si.columns[1], partner_si.name]])
    
    return  (trade_comp_log).diff().dropna()

In [14]:
trade_partners_index = pd.read_csv("cleaned data/Trading Partners 2000_2020.csv")
set_datetime_index(trade_partners_index)

hose_si_df = pd.DataFrame(hose[["Close"]])
hose_si_df.rename(columns = {"Close": "VNINDEX"}, inplace = True)

us_si = trade_partners_index["^GSPC"]
us_si.index =   us_si.index - pd.Timedelta(1, unit="day")

japan_si = trade_partners_index["^N225"]
hongkong_si = trade_partners_index["^HSI"]
korea_si = trade_partners_index["^KS11"]
shanghai_si = trade_partners_index["000001.SS"]

In [15]:
for partner_si in [us_si, japan_si, hongkong_si, korea_si, shanghai_si]:
    print(calc_trade_corr(hose_si_df, partner_si))

            ^GSPC   VNINDEX
^GSPC    1.000000  0.007002
VNINDEX  0.007002  1.000000
            ^N225   VNINDEX
^N225    1.000000  0.178233
VNINDEX  0.178233  1.000000
             ^HSI   VNINDEX
^HSI     1.000000  0.139768
VNINDEX  0.139768  1.000000
         ^KS11  VNINDEX
^KS11    1.000    0.118
VNINDEX  0.118    1.000
           000001.SS   VNINDEX
000001.SS   1.000000  0.122549
VNINDEX     0.122549  1.000000


In [ ]:
us_vn_rets = make_garch_df(hose_si_df, us_si)
us_vn_rets.head()

,VNINDEX,^GSPC
Date,,
2000-08-02,0.017860,0.010005
2000-08-07,0.033669,0.020605
2000-08-09,0.015959,-0.015325
2000-08-14,0.031173,0.016423
2000-08-16,0.015230,0.007811


In [ ]:
import mgarch

In [ ]:
vol = mgarch.mgarch()
vol.fit(us_vn_rets)
ndays = 10 # volatility of nth day
cov_nextday = vol.predict(ndays)

In [ ]:
cov_predict = []
for nday in len(us_vn_rets):
    cov_nextday.append

{'dist': 'norm',
 'cov': array([[ 2.93588699e-04, -1.09858054e-05],
        [-1.09858054e-05,  8.88996696e-05]])}

In [ ]:
print(vol.a)
print(vol.b)
print(vol.D_t)
print(vol.dof)

6.943343884604887e-05
0.10370417533200246
[[0.01723761 0.00975365]
 [0.01666773 0.00945725]
 [0.01785221 0.01000575]
 ...
 [0.01036467 0.0054509 ]
 [0.0101155  0.00546199]
 [0.0096354  0.00530213]]


AttributeError: 'mgarch' object has no attribute 'dof'

In [ ]:
vol.__dict__

{'dist': 'norm',
 'rt': matrix([[ 0.01723761,  0.00975365],
         [ 0.03304675,  0.02035354],
         [ 0.01533615, -0.0155757 ],
         ...,
         [ 0.01083251,  0.00620372],
         [ 0.00682668,  0.00108952],
         [-0.00239774,  0.00616708]]),
 'T': 3823,
 'N': 2,
 'mean': matrix([[0.00062262, 0.00025112]]),
 'D_t': array([[0.01723761, 0.00975365],
        [0.01666773, 0.00945725],
        [0.01785221, 0.01000575],
        ...,
        [0.01036467, 0.0054509 ],
        [0.0101155 , 0.00546199],
        [0.0096354 , 0.00530213]]),
 'a': 6.943343884604887e-05,
 'b': 0.10370417533200246}